# Imports

In [93]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set random seeds

In [94]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [95]:
raw_df = pd.read_csv("datasets/5_split/df_full.csv")

In [96]:
df = raw_df.copy()

# Separate features

In [97]:
# Dependent variables
labels = df.pop('very_good_health')

# CV folds
fold_ids = df.pop("fold_id_python")
folds = np.unique(fold_ids)

# Independent variables
features = df.drop(columns = ["fold_id_r"])

# Functions

## Get hyperparameters

In [98]:
def get_hyperparameters():
    no_of_layers = np.random.randint(1, 10)
    no_of_nodes = []
    for i in range(0, no_of_layers):
        no_of_nodes.append(np.random.randint(10, 100))
    learning_rate = np.random.uniform(0.0001, 0.01)
    epochs = np.random.randint(20, 500)
    patience = np.random.randint(5, 30)
    return no_of_layers, no_of_nodes, learning_rate, epochs, patience
    

## Build model

In [99]:
def build_model(train_features, no_of_layers, no_of_nodes, learning_rate):

    layers = []

    normaliser = keras.layers.Normalization(axis = -1)
    normaliser.adapt(np.array(train_features))
    layers.append(normaliser)

    for layer_no in range(no_of_layers):
        layers.append(keras.layers.Dense(no_of_nodes[layer_no], activation = "relu"))

    layers.append(keras.layers.Dense(1))     # Single output for regression value

    model = keras.Sequential(
        layers,
    )

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        loss='mse'
    )

    return model

## Build early stopper

In [100]:
def build_early_stopper(patience):
    early_stopper = keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        patience = patience,
        restore_best_weights = True,
        verbose = 1
    )
    return early_stopper

## Get evaluation metrics

In [101]:
def get_evaluation_metrics(val_labels, predictions):
    mae = mean_absolute_error(val_labels, predictions)
    mse = mean_squared_error(val_labels, predictions)
    r2 = r2_score(val_labels, predictions)
    return mae, mse, r2

# Cross-validation

In [ ]:
# Initialise evaluation results array
evaluation_results = []

# Outer loop for model evaluation will go here

# Hyperparameter selection loop
for i in range(1, 10):

    # Get hyperparameters
    no_of_layers, no_of_nodes, learning_rate, epochs, patience = get_hyperparameters()
    hps = {
        "no_of_layers": no_of_layers,
        "no_of_nodes": no_of_nodes,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "patience": patience
    }

    # Inner cross-validation
    for fold in folds:
        print(f"\n --- Training on fold {fold} ---")

        # Get training and validation sets - will be replaced by proper nested validation data splits
        is_in_validation_set = fold_ids == fold
        is_in_training_set = ~is_in_validation_set
        train_features = features.loc[is_in_training_set]
        train_labels = labels.loc[is_in_training_set]
        val_features = features.loc[is_in_validation_set]
        val_labels = labels.loc[is_in_validation_set]

        # Build model
        model = build_model(train_features, no_of_layers, no_of_nodes, learning_rate)
        early_stopper = build_early_stopper(patience)

        # Fit model
        model.fit(
            train_features,
            train_labels,
            epochs = epochs,
            validation_split = 0.2,     # Research whether I should use val_data for this, or whether that would lead to data leakage
            callbacks = [early_stopper]
        )

        # Get predictions using fitted model
        predictions = model.predict(val_features).flatten()

        # Get accuracy scores
        mae, mse, r2 = get_evaluation_metrics(val_labels, predictions)

        # Add scores for current fold to results
        evaluation_results.append({
            "hp_combination": i,
            "fold": fold,
            "hps": hps,
            "MAE": mae,
            "MSE": mse,
            "R2": r2
        })


 --- Training on fold 0 ---
Epoch 1/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0220 - val_loss: 0.0082
Epoch 2/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0033 - val_loss: 0.0041
Epoch 3/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0018 - val_loss: 0.0032
Epoch 4/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - val_loss: 0.0029
Epoch 5/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 0.0027
Epoch 6/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.5167e-04 - val_loss: 0.0026
Epoch 7/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.0753e-04 - val_loss: 0.0025
Epoch 8/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.8720e-04 - val_loss: 0.0025
Epoch 9/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.9316e-04 - val_loss: 0.0024
Epoch 10/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1212e-04 - val_loss: 0.0024
Epoch 11/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.4440e-04 - val_l

# Print results

In [ ]:
evaluation_results